# Convolutional Neural Network - Gap / Char Classification
Using TensorFlow

Also need to convert images in folder into csv data file

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

# Creating CSV
import glob
import csv

print("OpenCV: " + cv2.__version__)
print("Numpy: " + np.__version__)
print("TensorFlow: " + tf.__version__)

OpenCV: 3.1.0-dev
Numpy: 1.12.0
TensorFlow: 1.0.0-rc1


In [2]:
def implt(img, cmp=None, t=''):
    """ Show image using plt """
    plt.imshow(img, cmap=cmp)
    plt.title(t)
    plt.show()

## Load Images and Lables in CSV

In [3]:
# 755 chars and 306 gaps images, dimension 30x60 = 1800
# Or read_csv() form pandas
images = np.genfromtxt('data/gapdet/data.csv', delimiter=',')
labels = np.genfromtxt('data/gapdet/labels.csv', delimiter=',')
labels = np.reshape(labels, (len(labels), 1))

print("CSV data files loaded.")
print("Number of images: " + str(len(images)))

div = int(0.8 * len(images))

trainData = images[0:div]
trainLabels = labels[0:div]
evalData = images[div:]
evalLabels = labels[div:]
print("Training images: %g" % div)

CSV data files loaded.
Number of images: 5564
Training images: 4451


# Create classifier

### Simoid

In [4]:
# Place holders for data and labels
x = tf.placeholder(tf.float32, [None, 1800], name = "x")
y_ = tf.placeholder(tf.float32, [None, 1], name = "y_")

# Learned variables
W = tf.Variable(tf.random_normal([1800, 1]), name = "W")
b = tf.Variable(tf.random_normal([1]), name = "b")

y = tf.matmul(x, W) + b

# Training Model
cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=y, name="Cost"))
train_step = tf.train.AdamOptimizer(0.1).minimize(cross_entropy)

# Evaluating model
correct_prediction = tf.equal(tf.sigmoid(y) , y_)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# For saving and using model
activation = tf.sigmoid(tf.matmul(x, W) + b)
tf.add_to_collection("act", activation)

for i in range(5000):
#     sess.run(train_step, feed_dict={x: images, y_: labels})
    sess.run(train_step, feed_dict={x: trainData, y_: trainLabels})
    if i % 500 == 0:
        cost = cross_entropy.eval(feed_dict={x: trainData, y_: trainLabels})
        acc = accuracy.eval(feed_dict={x: trainData, y_: trainLabels})
        print("%d #  Cost: %g  Accuracy: %g" % (i, cost, acc))


# Saving model
# tf.train.export_meta_graph('models/CGClassifier')
saver = tf.train.Saver()
saver.save(sess, 'models/CGClassifier')
saver.export_meta_graph('models/CGClassifier.meta')


print("Eval accuracy: %g" % sess.run(accuracy, feed_dict={x: evalData, y_: evalLabels}))
print("Total accuracy: %g" % sess.run(accuracy, feed_dict={x: images, y_: labels}))

sess.close()

0 #  Cost: 1648.36  Accuracy: 0.69827
500 #  Cost: 71.91  Accuracy: 0.798922
1000 #  Cost: 16.5436  Accuracy: 0.873512
1500 #  Cost: 119.765  Accuracy: 0.761851
2000 #  Cost: 88.6834  Accuracy: 0.71557
2500 #  Cost: 14.4706  Accuracy: 0.911256
3000 #  Cost: 17.5025  Accuracy: 0.893507
3500 #  Cost: 3.97933  Accuracy: 0.921366
4000 #  Cost: 4.54229  Accuracy: 0.924736
4500 #  Cost: 6.75175  Accuracy: 0.928331
Eval accuracy: 0.632525
Total accuracy: 0.844177


### Sofmax

In [5]:
def sofmaxLabels(lab):
    """ Transform data for sofmax classification """
    newLab = np.zeros((len(lab), 2), dtype=np.int8)
    for i in range(len(lab)):
        newLab[i][int(lab[i][0])] = 1
    return newLab

llabels = sofmaxLabels(labels)
trainLabels = llabels[0:div]
evalLabels = llabels[div:]

In [6]:
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, [None, 1800])
y_ = tf.placeholder(tf.float32, [None, 2])

W = tf.Variable(tf.random_normal([1800, 2]))
b = tf.Variable(tf.random_normal([2]))

y = tf.matmul(x,W) + b

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.AdamOptimizer(0.1).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.global_variables_initializer())

for i in range(5000):
    sess.run(train_step, feed_dict={x: trainData, y_: trainLabels})
    if i % 500 == 0:
        cost = cross_entropy.eval(feed_dict={x: trainData, y_: trainLabels})
        acc = accuracy.eval(feed_dict={x: trainData, y_: trainLabels})
        print("%d #  Cost: %g  Accuracy: %g" % (i, cost, acc))

    

print("Eval accuracy: %g" % accuracy.eval(feed_dict={x: evalData, y_: evalLabels}))
print("Total accuracy: %g" % accuracy.eval(feed_dict={x: images, y_: llabels}))

sess.close()

0 #  Cost: 3676.05  Accuracy: 0.700067
500 #  Cost: 29.4448  Accuracy: 0.897326
1000 #  Cost: 58.508  Accuracy: 0.864075
1500 #  Cost: 15.2268  Accuracy: 0.929903
2000 #  Cost: 28.2934  Accuracy: 0.910133
2500 #  Cost: 42.1843  Accuracy: 0.89171
3000 #  Cost: 90.6883  Accuracy: 0.836217
3500 #  Cost: 16.3811  Accuracy: 0.933498
4000 #  Cost: 18.9064  Accuracy: 0.902269
4500 #  Cost: 19.8118  Accuracy: 0.884071
Eval accuracy: 0.72327
Total accuracy: 0.853343


### Convulation Neural Network

In [7]:
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, [None, 1800])
y_ = tf.placeholder(tf.float32, [None, 2])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x_image = tf.reshape(x, [-1, 30, 60, 1])

# 1. Layer
W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 12], stddev=0.1), name="W_conv1")
b_conv1 = tf.Variable(tf.constant(0.1, shape=[12]), name="b_conv1")

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# 2. Layer
W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 12, 24], stddev=0.1), name="W_conv2")
b_conv2 = tf.Variable(tf.constant(0.1, shape=[24]), name="b_conv2")

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Connected layer
W_fc1 = tf.Variable(tf.truncated_normal([8 * 15 * 24, 512], stddev=0.1), name="W_fc1")
b_fc1 = tf.Variable(tf.constant(0.1, shape=[512]), name="b_fc1")

h_pool2_flat = tf.reshape(h_pool2, [-1, 8*15*24])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Output layer
W_fc2 = tf.Variable(tf.truncated_normal([512, 2], stddev=0.1), name="W_fc2")
b_fc2 = tf.Variable(tf.constant(0.1, shape=[2]), name="b_fc2")

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


# Train
# Stacking on 18029.2 cost
# cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv + 1e-10))
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()

for i in range(300):
    if i%10 == 0:
        cost = cross_entropy.eval(feed_dict={x: trainData, y_: trainLabels, keep_prob: 1.0})
        acc = accuracy.eval(feed_dict={x: trainData, y_: trainLabels, keep_prob: 1.0})
        print("%d #  Cost: %g  Accuracy: %g" % (i, cost, acc))
        
    if i%50 == 0:
        saver.save(sess, 'models/CNN-CG')
        saver.export_meta_graph('models/CNN-CG.meta')

    train_step.run(feed_dict={x: trainData, y_: trainLabels, keep_prob: 0.5})

print("Eval accuracy %g" % accuracy.eval(feed_dict={x: evalData, y_: evalLabels, keep_prob: 1.0}))
print("Total accuracy: %g" % accuracy.eval(feed_dict={x: images, y_: llabels, keep_prob: 1.0}))

sess.close()

0 #  Cost: 169.092  Accuracy: 0.685913
10 #  Cost: 71.9017  Accuracy: 0.700966
20 #  Cost: 42.2531  Accuracy: 0.744327
30 #  Cost: 29.0836  Accuracy: 0.779151
40 #  Cost: 22.4769  Accuracy: 0.800494
50 #  Cost: 17.7913  Accuracy: 0.818468
60 #  Cost: 14.0565  Accuracy: 0.830375
70 #  Cost: 11.639  Accuracy: 0.844529
80 #  Cost: 9.43386  Accuracy: 0.852842
90 #  Cost: 7.74464  Accuracy: 0.862503
100 #  Cost: 6.62132  Accuracy: 0.869692
110 #  Cost: 5.17878  Accuracy: 0.880926
120 #  Cost: 4.30363  Accuracy: 0.891036
130 #  Cost: 3.65433  Accuracy: 0.897551
140 #  Cost: 3.03407  Accuracy: 0.903617
150 #  Cost: 2.52885  Accuracy: 0.913503
160 #  Cost: 1.98134  Accuracy: 0.925859
170 #  Cost: 1.70483  Accuracy: 0.929903
180 #  Cost: 1.49411  Accuracy: 0.934621
190 #  Cost: 1.17387  Accuracy: 0.940687
200 #  Cost: 0.954691  Accuracy: 0.948551
210 #  Cost: 0.803997  Accuracy: 0.95237
220 #  Cost: 0.656812  Accuracy: 0.956414
230 #  Cost: 0.522657  Accuracy: 0.961582
240 #  Cost: 0.416346  Ac